# FIRE Community on Reddit

What is the FU number the fire community is aiming maybe by age, location, and over time?

## Reddit Scrape

Community: r/FIRE \
thread: /What_is_your_fire_number/ \
Code Reference: with help from ChatGPT https://chatgpt.com/ and https://colab.research.google.com/


**Privacy note:**

*   Credentials were created for the class but might have to figure out how to hide
*   Username will also have to be masked to be anonymous

In [ ]:
#!pip install requests beautifulsoup4

In [ ]:
# Library
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.auth import HTTPBasicAuth
import json
import csv
from os import read

In [ ]:
# Reddit API credentials
client_id = '8-xAlbBmc1_FYraSwJENtw'
client_secret = 'Jg9JyK9R_ixyNESMaGE3BwLzmBhf2w'
username = 'Western_Signal4499'
password = 'ADS509finAl'
user_agent = 'ADS509Scrapper/0.1 by Western_Signal4499'

# Get the OAuth token
auth = HTTPBasicAuth(client_id, client_secret)
headers = {'User-Agent': user_agent}
data = {'grant_type': 'password', 'username': username, 'password': password}

response = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, headers=headers, data=data)

if response.status_code == 200:
    token = response.json()['access_token']
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Use the token to search for threads
headers['Authorization'] = f'bearer {token}'
search_url = "https://oauth.reddit.com/r/FIRE/search?q=what%20is%20your%20fire%20number&restrict_sr=1"
search_response = requests.get(search_url, headers=headers)

if search_response.status_code == 200:
    search_data = search_response.json()
    threads = search_data['data']['children']

    # Prepare CSV
    with open('fire_comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Thread ID', 'Title', 'User', 'Comment'])

        for thread in threads:
            thread_title = thread['data']['title']
            thread_id = thread['data']['id']
            comments_url = f"https://oauth.reddit.com/r/FIRE/comments/{thread_id}.json"

            # Retrieve comments for the current thread
            comments_response = requests.get(comments_url, headers=headers)
            if comments_response.status_code == 200:
                comments_data = comments_response.json()
                comments = comments_data[1]['data']['children']

                for comment in comments:
                    if comment['kind'] == 't1':
                        username = comment['data']['author']
                        body = comment['data']['body']
                        writer.writerow([thread_id, thread_title, username, body])

                print(f"Thread ID: {thread_id}, Title: {thread_title}, User1: {username}, Comment: {body}")

            else:
                print(f"Failed to retrieve comments for thread ID {thread_id} with status code {comments_response.status_code}")

else:
    print(f"Failed to retrieve search results with status code {search_response.status_code}")

Thread ID: 1fqzb0p, Title: What is your fire number? , User: nybhh, Comment: $5 million.
Thread ID: rfxxkz, Title: What is your fire number?, User: xmxprztm, Comment: 4M at least. Going to move to Spain
Thread ID: zddhma, Title: For folks who plan to FIRE in HCOL / VHCOL areas, what is your FIRE number? Why not move to MCOL / LCOL area?, User: [deleted], Comment: Some services or experiences are only available in hcol areas. Good fine dining options and/or weather and or ___.
Thread ID: 10ex17r, Title: What is your FIRE number?, User: zenjdg, Comment: My ideal scenario is $100k per month in tax-free cash flow.
Thread ID: z5a29v, Title: What is your fire number and how did you arrive at it?, User: [deleted], Comment: [deleted]
Thread ID: 1ef8jjf, Title: How to split finances when one is FIRE'd and one isn't FIRE'ing, User: [deleted], Comment: Break up
Thread ID: oh6i17, Title: What is your FIRE number and at what age do you want to achieve it by?, User: FIREDOC627, Comment: Wife is 8 ye

In [ ]:
rfire_df = pd.read_csv('fire_comments.csv')
rfire_df.head()

,Thread ID,Title,User,Comment
0,1fqzb0p,What is your fire number?,wowsocool4u,$1.9m. Targeting a few months after i turn 50...
1,1fqzb0p,What is your fire number?,manimopo,1.5m with paid off houses
2,1fqzb0p,What is your fire number?,Zphr,Our target back in 2014 was $1.2M and a paid-o...
3,1fqzb0p,What is your fire number?,Animag771,I'm just waiting for someone to say something ...
4,1fqzb0p,What is your fire number?,Aromatic_Heart,"On days I hate my job, 600k. The other days, d..."


In [ ]:
rfire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Thread ID  1458 non-null   object
 1   Title      1458 non-null   object
 2   User       1458 non-null   object
 3   Comment    1458 non-null   object
dtypes: object(4)
memory usage: 45.7+ KB


In [ ]:
rfire_df['Title'].unique()

array(['What is your fire number? ', 'What is your fire number?',
       'For folks who plan to FIRE in HCOL / VHCOL areas, what is your FIRE number? Why not move to MCOL / LCOL area?',
       'What is your FIRE number?',
       'What is your fire number and how did you arrive at it?',
       "How to split finances when one is FIRE'd and one isn't FIRE'ing",
       'What is your FIRE number and at what age do you want to achieve it by?',
       'We’re all privileged',
       'How my wife and I will retire before 50; neither of us make over $75K',
       '10 tips from my 20 year journey to financial freedom',
       'Is toxic corporate culture why most of us want to Fire? ',
       'The cliff is steep: How do you choose to retire early when working 5 more years can bring so much more',
       'How did you come up with your FIRE number? What is the best calculator, way to get at it?',
       'Forgive me if this is too personal, but what was your estimated net worth around 18 and how did 